In [4]:
# ==============================================================
# FAST & ROBUST ECC TO EXCEL CONVERTER (FIXED REGEX)
# ==============================================================
from google.colab import files
import pandas as pd
import re
import os
import time

print("--- Step 1: Configuration ---")

# 1. Ask for Start ID
default_id = 50
start_id_input = input(f"Enter Linklist Start Number (default {default_id}): ")
try:
    current_id = int(start_id_input)
except ValueError:
    current_id = default_id
print(f"-> Using Start ID: {current_id}")

# 2. Upload File
print("\n--- Step 2: Upload File ---")
print("Please upload your 'ExportECC' file (CSV or TXT)...")
uploaded = files.upload()

if not uploaded:
    print("X No file uploaded. Please run the cell again.")
else:
    start_time = time.time()
    filename = next(iter(uploaded))
    print(f"\n-> Processing: {filename}...")

    # --- PART A: ROBUST FILE READING ---
    # Read raw bytes to handle any encoding issues
    raw_data = uploaded[filename]

    # Try UTF-8 with BOM (common in Excel CSVs), then Latin-1
    try:
        content = raw_data.decode('utf-8-sig', errors='ignore')
    except:
        content = raw_data.decode('latin-1', errors='ignore')

    # Clean hidden null bytes (common in generated files)
    content = content.replace('\x00', '')
    lines = content.splitlines()

    # --- PART B: FAST PARSING (REGEX) ---
    collections = []
    unit_map = {}

    # Regex 1: Matches the Collection Name at the start of the line
    # Looks for: Starts with #, captures everything until the first semicolon
    name_pattern = re.compile(r"^#([^;]+)")

    # Regex 2: Matches Codes and Quantities anywhere in the line
    # Looks for: #, then (Code), then [, then (Qty), then ]
    # [^\[;#]+ means "any character that is NOT [ or ; or #"
    code_pattern = re.compile(r"#([^\[;#]+)\[([\d\.,]+)\]")

    row_count = 0

    for line in lines:
        line = line.strip()
        if not line: continue

        # 1. Check if it is a Collection Line (Starts with #)
        if line.startswith("#"):
            # A. Extract Name
            name_match = name_pattern.search(line)
            if name_match:
                current_coll_name = name_match.group(1).strip()
            else:
                # Fallback if regex fails but line starts with #
                parts = line.split(";")
                current_coll_name = parts[0].replace("#", "").strip()

            # B. Extract All Codes at once (Faster)
            found_codes = code_pattern.findall(line)

            # If no codes found, we assume 0 units but still increment ID?
            # (Adjust logic here if you want empty rows for empty collections)

            for code, qty_str in found_codes:
                # Clean Qty (replace comma with dot for decimals)
                qty_str = qty_str.replace(",", ".")
                try:
                    qty = float(qty_str)
                    loop_count = int(qty) if qty.is_integer() else 1
                except:
                    loop_count = 1

                # Add Rows
                for _ in range(loop_count):
                    collections.append({
                        "Linklist ID": current_id,
                        "Linklist Name": current_coll_name,
                        "Linklist Units": code.strip(),
                        "Number of Units": "[1.000]",
                        "Series ID": 0
                    })
                    row_count += 1

            # Increment ID for the next collection found
            current_id += 1

        # 2. Check if it is a Unit Line (Starts with ;#)
        elif line.startswith(";#"):
            # Clean: ;#UnitName; -> UnitName
            unit_name = line.replace(";", "").replace("#", "").strip()

            if unit_name:
                # Link to the PREVIOUS collection ID
                active_id = current_id - 1

                if unit_name not in unit_map:
                    unit_map[unit_name] = []
                unit_map[unit_name].append(active_id)

    # --- PART C: BUILD EXCEL ---
    print(f"-> Parsing complete. Generating Excel with {row_count} collection rows...")

    df = pd.DataFrame(collections)

    # Ensure Left Columns Exist (Fix for blank columns)
    req_cols = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID"]
    for c in req_cols:
        if c not in df.columns:
            df[c] = "" # Fill with empty string if no data found

    # Prepare Right Columns (Units)
    unit_data_list = []
    for u_name, id_list in unit_map.items():
        # Build Linked List String: <1~1~50|2~1~51|>
        link_str = "<"
        for idx, linked_id in enumerate(id_list):
            link_str += f"{idx+1}~1~{linked_id}|"
        link_str += ">"

        unit_data_list.append({
            "Unit Name": u_name,
            "Linked Lists": link_str
        })

    df_units = pd.DataFrame(unit_data_list)

    # Merge Side-by-Side (Reset index to ensure they start at top)
    final_df = pd.concat([df.reset_index(drop=True), df_units.reset_index(drop=True)], axis=1)

    # Final Column Order & Fill NaNs
    final_cols = ["Linklist ID", "Linklist Name", "Linklist Units", "Number of Units", "Series ID", "Unit Name", "Linked Lists"]

    # Add any missing columns to final DF
    for c in final_cols:
        if c not in final_df.columns:
            final_df[c] = ""

    final_df = final_df[final_cols]

    # Fill any remaining NaNs (e.g., if one side is shorter than the other)
    final_df = final_df.fillna("")

    # --- PART D: DOWNLOAD ---
    output_filename = os.path.splitext(filename)[0] + ".xlsx"
    final_df.to_excel(output_filename, index=False)

    end_time = time.time()
    print(f"\n[Success] File generated in {round(end_time - start_time, 2)} seconds.")
    print(f"Downloading {output_filename}...")
    files.download(output_filename)

--- Step 1: Configuration ---
Enter Linklist Start Number (default 50): 50
-> Using Start ID: 50

--- Step 2: Upload File ---
Please upload your 'ExportECC' file (CSV or TXT)...


Saving MarmolineExportECC.csv to MarmolineExportECC (3).csv

-> Processing: MarmolineExportECC (3).csv...
-> Parsing complete. Generating Excel with 684 collection rows...

[Success] File generated in 0.8 seconds.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>